In [1]:
import pandas as pd
import numpy as np

from util.constant import *
%load_ext autoreload
%autoreload 2

In [2]:
from util.scan_database import ExtractController, scan_dir

In [3]:
data_path = 'data'

In [4]:
import sqlite3
from util.db_action import get_all_table_names, get_all_information_from_table_as_pd_dataframe
from collections import defaultdict
tot_data = defaultdict(dict)
inc = 0

for d in scan_dir(data_path):
    if d.endswith('.rar') or d.endswith('.zip'):
        with ExtractController(d) as extracted_dir:
            for t in scan_dir(extracted_dir):
                if t.endswith('.db'):
                    con = sqlite3.connect(t)
                    table_names = get_all_table_names(con)
                    for n in table_names:
                        df = get_all_information_from_table_as_pd_dataframe(con, n)
                        tot_data[inc][n] = df
        inc += 1

In [5]:
def get_one_table(n):
    return {i:d[n] for i, d in tot_data.items()}

# content

In [6]:
def count_cols_in_one_table(table_name, cols):
    contents = get_one_table(table_name)
    def count_times(n, c):
        n = n[c]
        m = set(n)
        return {a: (n==a).sum() for a in m}
    return [{i: count_times(t, c) for i, t in contents.items()} for c in cols]

In [7]:
#content的各项操作的个数
count_cols_in_one_table(CONTENT_INFO, ['operation'])

[{}]

In [8]:
# content的相邻两次操作之间的时间间隔
def get_delta_time(n, col_name='time'):
    from util.utility import string_to_datetime, timedelta_milliseconds
    is_new = True
    pre=None
    dts=[]
    for t in n.iterrows():
        t=t[1]
        if is_new:
            pre=string_to_datetime(t[col_name])
            is_new=False
        else:
            dts.append(timedelta_milliseconds(string_to_datetime(t[col_name])-pre))
            pre = string_to_datetime(t[col_name])
    return pd.Series(dts)

contents = get_one_table(CONTENT_INFO)
for i, ii in contents.items():
    m = get_delta_time(ii)
    print("id: {}".format(i))
    print("total time:{}".format(m.sum()))
    print(m.describe())

# COMMAND TEXT

In [9]:
#text commmand 各个操作的个数
count_cols_in_one_table(COMMAND_TEXT, ['action'])

[{}]

# BREAK_POINT

In [10]:
#断点的个数
{i: len(j)  for i, j in count_cols_in_one_table(BREAK_POINT, ['id'])[0].items()}

{}

# DEBUG_INFO

In [11]:
#debug时停顿原因的统计
count_cols_in_one_table(DEBUG_INFO,['type'])

[{}]

In [12]:
def cal_time_delta(n):
    from util.utility import string_to_datetime, timedelta_milliseconds
    import datetime
    zero_time = timedelta_milliseconds(datetime.timedelta(microseconds=0))
    ts_list = []
    n_list = []
    c = 0
    is_new = True
    pre_time = None
    for t in n.iterrows():
        t=t[1]
        if t['type'] == 'run':
            if not is_new:
                c += 1
            else:
                is_new = False
            ts_list.append(zero_time)
            n_list.append(c)
            pre_time = string_to_datetime(t['timestamp'])
        else:
            ts_list.append(timedelta_milliseconds(string_to_datetime(t['timestamp'])-pre_time))
            pre_time = string_to_datetime(t['timestamp'])
            n_list.append(c)
    return pd.DataFrame({'seq_id': n_list, 'delta_time': ts_list})

In [13]:
# 单次debug的时间
# 单次debug的长度
debug_info = get_one_table(DEBUG_INFO)
for i, ii in debug_info.items():
    m = cal_time_delta(ii)
    print("id: {}".format(i))
    print("mean miliseconds: {}".format(m.groupby('seq_id')['delta_time'].sum().describe()))
    print("run count: {}".format(m.groupby('seq_id')['delta_time'].count().describe()))
    print("")